# Str Output parser

In [8]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
load_dotenv()
from langchain_ollama import ChatOllama
model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser
result = chain.invoke({'topic':'black hole'})

print(result)


Here is a 5-line summary of the text:

A black hole is a region in space where gravity is so strong that nothing, including light, can escape. It is formed when a massive star collapses and its gravity warps spacetime around it, creating an event horizon that marks the point of no return. Black holes have several characteristics, including a singularity at their center, incredibly strong gravitational fields, and no emission of radiation or light. There are four types of black holes, ranging from stellar to supermassive, each with unique properties and effects on spacetime. Continued research and observation of black holes will help us better understand these enigmatic objects and their role in the universe.


# JSON output parser

In [36]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser ,JsonOutputParser
from dotenv import load_dotenv
load_dotenv()

parser = JsonOutputParser()
model = ChatOllama(
    model="qwen3:latest",
    temperature=0.7
)
template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

In [37]:
template.format(topic='black hole')

'Give me 5 facts about black hole \n Return a JSON object.'

In [38]:
type(result)

dict

# Structured Output Parser

In [19]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from langchain_ollama import ChatOllama
load_dotenv()

# Define the model
model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)



schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

ModuleNotFoundError: No module named 'langchain.output_parsers'

# Pydantic Output Parser

In [44]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
load_dotenv()

# Define the model
model = ChatOllama(
    model="qwen3:latest",
    temperature=0.7
)
class Person(BaseModel):
    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

final_result = chain.invoke({'place':'canada'})

print(final_result)

name='Emily Carter' age=25 city='Vancouver'


In [27]:
final_result.name, final_result.age, final_result.city

('Anura Wijeyasinghe', 25, 'Kandy')

# Nested pydantic Models

In [ ]:
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain_ollama import ChatOllama
load_dotenv()

# Define the model
model = ChatOllama(
    model="qwen3:latest",
    temperature=0.7
)

c:\Users\prana\Desktop\VS_CODE\LangChainModels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
class Subtopic(BaseModel):
    name: str = Field(description="Name of the model type (e.g., Vision Models, Text Models)") 
    advantages: List[str] = Field(description="List of advantages of this model type") 
    disadvantages: List[str] = Field(description="List of disadvantages of this model type") 
    conclusion: str = Field(description="Short conclusion about this model type") 
    
class Topic(BaseModel):
    title: str = Field(description="Main topic name") 
    intro: str = Field(description="Definition or introduction of the topic")
    categories: List[Subtopic] = Field(description="Different model types under this topic")

In [30]:
parser = PydanticOutputParser(pydantic_object=Topic)

template = PromptTemplate(
    template='Generate a report on : {topic}\n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

response = chain.invoke({'topic':'LLMs'})

print(response)

title='Large Language Models (LLMs)' intro='Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to perform a wide range of tasks, including text generation, translation, summarization, and question-answering. They leverage deep learning techniques to understand and generate human-like text.' categories=[Subtopic(name='Text Models', advantages=['Natural language understanding and generation', 'Versatile for various text-based tasks', 'High accuracy in language-related tasks'], disadvantages=['Limited to text processing, lacking visual/audio capabilities', 'Potential for generating biased or inaccurate content', 'Requires extensive training data for optimal performance'], conclusion='Text models are essential for language-based applications but have limitations in handling non-text data.'), Subtopic(name='Vision Models', advantages=['Capable of processing and understanding visual data', 'Effective in image recognition and descrip

In [ ]:
class Subtopic(BaseModel):
    name: str = Field(description="Name of the model type (e.g., Vision Models, Text Models)") 
    advantages: List[str] = Field(description="List of advantages of this model type") 
    disadvantages: List[str] = Field(description="List of disadvantages of this model type") 
    conclusion: str = Field(description="Short conclusion about this model type") 
    
class Topic(BaseModel):
    title: str = Field(description="Main topic name") 
    intro: str = Field(description="Definition or introduction of the topic")
    categories: List[Subtopic] = Field(description="Different model types under this topic")

In [31]:
print(f"""
Topic: {response.title}

Introduction:
{response.intro}
""")

for category in response.categories:
    print(f"Model Type: {category.name}")
    
    print("Advantages:")
    for adv in category.advantages:
        print(f"  - {adv}")
    
    print("Disadvantages:")
    for disadv in category.disadvantages:
        print(f"  - {disadv}")
    
    print(f"Conclusion: {category.conclusion}")
    print("\n" + "-"*50 + "\n")


Topic: Large Language Models (LLMs)

Introduction:
Large Language Models (LLMs) are advanced artificial intelligence systems trained on vast amounts of text data to perform a wide range of tasks, including text generation, translation, summarization, and question-answering. They leverage deep learning techniques to understand and generate human-like text.

Model Type: Text Models
Advantages:
  - Natural language understanding and generation
  - Versatile for various text-based tasks
  - High accuracy in language-related tasks
Disadvantages:
  - Limited to text processing, lacking visual/audio capabilities
  - Potential for generating biased or inaccurate content
  - Requires extensive training data for optimal performance
Conclusion: Text models are essential for language-based applications but have limitations in handling non-text data.

--------------------------------------------------

Model Type: Vision Models
Advantages:
  - Capable of processing and understanding visual data
  